#### Read data from source to DataFrame in local Spark setup and display DataFrame schema.

In [129]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *


In [130]:
# spark = SparkSession.builder.appName("PySpark Assignment").getOrCreate()
spark = SparkSession.builder.getOrCreate()

titanic = spark.read.option("header", "true").option("inferSchema", "true").option("header", "false").csv("data/titanic.csv")
# titanic.printSchema()

In [131]:

columns = ["PassengerId","Survived","Pclass","Name","Sex","Age","SibSp","Parch","Ticket","Fare","Cabin","Embarked","Timestamp"]
titanic = titanic.toDF(*columns)

# Option 2: Use withColumnRenamed() method to rename existing columns
for i, col_name in enumerate(titanic.columns):
    titanic = titanic.withColumnRenamed(col_name, columns[i])
titanic.show()
titanic.printSchema()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|          Timestamp|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|2020-01-01 13:45:25|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|2020-01-01 13:44:48|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|2020-01-01 13:38:11|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|2020-01-01 13:32:00|
|          5|       0|     3|Allen, Mr. Willia...|  mal

In [132]:
column_types = titanic.dtypes
column_types

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'int'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string'),
 ('Timestamp', 'timestamp')]

#### For numerical columns, calculate minimum, maximum and average values.

In [133]:
numerical_columns = [column[0] for column in column_types if column[1] in ['int', 'float']]
summary_df = titanic.describe(numerical_columns)
summary_df.select(numerical_columns).summary("max","min","mean").show()


+-------+------------------+------------------+------------------+------------------+------------------+-----------------+
|summary|       PassengerId|          Survived|            Pclass|               Age|             SibSp|            Parch|
+-------+------------------+------------------+------------------+------------------+------------------+-----------------+
|    max|               891|               891|               891|                80|               891|              891|
|    min|                 1|                 0|0.8360712409770491|                 0|                 0|                0|
|   mean|497.27076840304596|178.57408616762064|179.62894264325715|167.64315089562422|180.12515025772694|179.6375301872114|
+-------+------------------+------------------+------------------+------------------+------------------+-----------------+



#### For categorical columns, create and apply UDF that will change the last letter of every word to “1”.

In [134]:
str_columns = ["Sex","Cabin","Embarked"]
def change_last_letter_after_space(word):
    if word is not None:
        words = word.split()
        for i in range(len(words)):
            words[i] = words[i][:-1] + "1"
        return " ".join(words)
    return word

change_last_letter_udf = udf(change_last_letter_after_space, StringType())


for column in str_columns:
    titanic = titanic.withColumn(column, change_last_letter_udf(titanic[column]))

titanic.show()


+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|          Timestamp|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------------------+
|          1|       0|     3|Braund, Mr. Owen ...|  mal1|  22|    1|    0|       A/5 21171|   7.25| null|       1|2020-01-01 13:45:25|
|          2|       1|     1|Cumings, Mrs. Joh...|femal1|  38|    1|    0|        PC 17599|71.2833|  C81|       1|2020-01-01 13:44:48|
|          3|       1|     3|Heikkinen, Miss. ...|femal1|  26|    0|    0|STON/O2. 3101282|  7.925| null|       1|2020-01-01 13:38:11|
|          4|       1|     1|Futrelle, Mrs. Ja...|femal1|  35|    1|    0|          113803|   53.1| C121|       1|2020-01-01 13:32:00|
|          5|       0|     3|Allen, Mr. Willia...|  mal

#### Sort DataFrame by the first column and save the results to the Parquet file.

In [135]:

sorted_df = titanic.orderBy(col(titanic.columns[0]))
sorted_df.write.mode("overwrite").parquet("output/titanic_data.parquet")